In [5]:
library(tidyverse)
library(tidymodels)
library(repr)
library(ggplot2)
library(lubridate)
library(forcats)
library(tidyr)

In [6]:
players <- read_csv("data/players.csv")

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [7]:
players_cleaned <- players |> filter(!is.na(experience), !is.na(subscribe), !is.na(hashedEmail), !is.na(played_hours), !is.na(name), !is.na(gender), !is.na(Age))

In [25]:
player_sessions <- players_cleaned |> group_by(hashedEmail) |> summarize(number_of_sessions = n(), .groups = "drop")
player_combined <- players_cleaned |> left_join (player_sessions, by = "hashedEmail") |> filter (!played_hours == 0) |> select(experience, subscribe, played_hours, number_of_sessions) |> mutate(experience = as.factor(experience), subscribe = as.factor(subscribe))
player_combined

experience,subscribe,played_hours,number_of_sessions
<fct>,<fct>,<dbl>,<int>
Pro,TRUE,30.3,1
Veteran,TRUE,3.8,1
Amateur,TRUE,0.7,1
Regular,TRUE,0.1,1
Amateur,TRUE,0.1,1
Veteran,TRUE,1.6,1
Beginner,TRUE,1.5,1
Amateur,TRUE,0.2,1
Amateur,TRUE,48.4,1


mutate-joins {dplyr},R Documentation
"x, y","A pair of data frames, data frame extensions (e.g. a tibble), or lazy data frames (e.g. from dbplyr or dtplyr). See Methods, below, for more details."
by,"A join specification created with join_by(), or a character vector of variables to join by. If NULL, the default, ⁠*_join()⁠ will perform a natural join, using all variables in common across x and y. A message lists the variables so that you can check they're correct; suppress the message by supplying by explicitly. To join on different variables between x and y, use a join_by() specification. For example, join_by(a == b) will match x$a to y$b. To join by multiple variables, use a join_by() specification with multiple expressions. For example, join_by(a == b, c == d) will match x$a to y$b and x$c to y$d. If the column names are the same between x and y, you can shorten this by listing only the variable names, like join_by(a, c). join_by() can also be used to perform inequality, rolling, and overlap joins. See the documentation at ?join_by for details on these types of joins. For simple equality joins, you can alternatively specify a character vector of variable names to join by. For example, by = c(""a"", ""b"") joins x$a to y$a and x$b to y$b. If variable names differ between x and y, use a named character vector like by = c(""x_a"" = ""y_a"", ""x_b"" = ""y_b""). To perform a cross-join, generating all combinations of x and y, see cross_join()."
copy,"If x and y are not from the same data source, and copy is TRUE, then y will be copied into the same src as x. This allows you to join tables across srcs, but it is a potentially expensive operation so you must opt into it."
suffix,"If there are non-joined duplicate variables in x and y, these suffixes will be added to the output to disambiguate them. Should be a character vector of length 2."
...,Other parameters passed onto methods.
keep,"Should the join keys from both x and y be preserved in the output? If NULL, the default, joins on equality retain only the keys from x, while joins on inequality retain the keys from both inputs. If TRUE, all keys from both inputs are retained. If FALSE, only keys from x are retained. For right and full joins, the data in key columns corresponding to rows that only exist in y are merged into the key columns from x. Can't be used when joining on inequality conditions."
na_matches,"Should two NA or two NaN values match? ""na"", the default, treats two NA or two NaN values as equal, like %in%, match(), and merge(). ""never"" treats two NA or two NaN values as different, and will never match them together or to any other values. This is similar to joins for database sources and to base::merge(incomparables = NA)."
multiple,"Handling of rows in x with multiple matches in y. For each row of x: ""all"", the default, returns every match detected in y. This is the same behavior as SQL. ""any"" returns one match detected in y, with no guarantees on which match will be returned. It is often faster than ""first"" and ""last"" if you just need to detect if there is at least one match. ""first"" returns the first match detected in y. ""last"" returns the last match detected in y."
unmatched,"How should unmatched keys that would result in dropped rows be handled? ""drop"" drops unmatched keys from the result. ""error"" throws an error if unmatched keys are detected. unmatched is intended to protect you from accidentally dropping rows during a join. It only checks for unmatched keys in the input that could potentially drop rows. For left joins, it checks y. For right joins, it checks x. For inner joins, it checks both x and y. In this case, unmatched is also allowed to be a character vector of length 2 to specify the behavior for x and y independently."
relationship,"Handling of the expected relationship between the keys of x and y. If the expectations chosen from the list below are invalidated, an error is thrown. NULL, the default, doesn't expect there to be an

In [10]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 5) |> set_engine("kknn") |> set_mode("classification")
knn_fit <- knn_spec |> fit(subscribe ~ experience + played_hours + number_of_sessions, data = player_combined)
player_recipe <- recipe(subscribe ~ experience + played_hours + number_of_sessions, data = player_combined) |> step_scale(played_hours)|>
step_center(played_hours) |> prep()
scaled_player <- bake(player_recipe, player_combined)
head(scaled_player)

experience,played_hours,number_of_sessions,subscribe
<fct>,<dbl>,<int>,<fct>
Pro,0.85603962,1,TRUE
Veteran,-0.07385234,1,TRUE
Veteran,-0.20719534,1,FALSE
Amateur,-0.18263216,1,TRUE
Regular,-0.20368631,1,TRUE
Amateur,-0.20719534,1,TRUE
